# TF高级API - estimator

更多高级API参考
https://github.com/GoogleCloudPlatform/tf-estimator-tutorials

In [1]:
# -*- coding: utf-8 -*-

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from six.moves import xrange

In [2]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('../MNIST_data', one_hot=False)

classes_num = 10

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def model_fn(features, labels, mode):
    features = features['images']
    with tf.name_scope('dense1'):
        dense1 = tf.layers.dense(features, 128, activation=tf.nn.relu)
        dense1_do = tf.nn.dropout(dense1, keep_prob=0.9)
    with tf.name_scope('dense2'):
        dense2 = tf.layers.dense(dense1_do, classes_num, activation=tf.nn.relu)
        dense2_do = tf.nn.dropout(dense2, keep_prob=0.9)

    predictions = tf.argmax(tf.nn.softmax(dense2_do), axis=-1)

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    labels = tf.cast(labels, dtype=tf.int32)
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=dense2_do, labels=labels))
    optimizer_op = tf.train.AdamOptimizer(1e-3).minimize(loss_op, global_step=tf.train.get_global_step())
    accuracy_op = tf.metrics.accuracy(labels=labels, predictions=predictions)

    esimator_spec = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        loss=loss_op,
        train_op=optimizer_op,
        eval_metric_ops={'accuracy': accuracy_op})
    return esimator_spec

In [4]:
# model_dir: Directory to save model parameters, graph and etc. This can
#            also be used to load checkpoints from the directory into a estimator to
#            continue training a previously saved model.
model = tf.estimator.Estimator(model_fn, model_dir='./model')
tf.logging.set_verbosity(tf.logging.INFO)

# train
input_fn = tf.estimator.inputs.numpy_input_fn(x={'images': mnist.train.images}, y=mnist.train.labels,
                                              batch_size=1000, num_epochs=None, shuffle=True)
model.train(input_fn, steps=15)

# evaluate
test_fn = tf.estimator.inputs.numpy_input_fn(x={'images': mnist.test.images}, y=mnist.test.labels,
                                             batch_size=1000, num_epochs=None, shuffle=False)
eva = model.evaluate(test_fn, steps=5)
for key, value in eva.items():
    print(key, value)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_train_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4edec8f390>, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_session_config': None, '_is_chief': True, '_tf_random_seed': None, '_num_ps_replicas': 0, '_master': '', '_save_summary_steps': 100, '_evaluation_master': '', '_model_dir': './model', '_task_id': 0, '_save_checkpoints_secs': 600, '_service': None, '_num_worker_replicas': 1, '_keep_checkpoint_max': 5, '_global_id_in_cluster': 0, '_task_type': 'worker'}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model/model.ckpt-20
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 21 into .